In [2]:
conda activate venv


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tensorflow-intel==2.13.1 from https://files.pythonhosted.org/packages/6b/f8/3dc773923f4afa73b894ba25f83f8a0f1a549783ab99b496e0f5d0b86099/tensorflow_intel-2.13.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.2 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [3]:
import pandas as pd
import torch
import tensorflow as tf
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments

# Read the CSV file
df = pd.read_csv("C:\\testing\\Final_phonetics_T5.csv")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is NOT available")
    
# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("tfGPU is available")
else:
    print("tfGPU is NOT available")

# Initialize tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")# Initialize lists to store formatted data


GPU is NOT available
tfGPU is NOT available


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Initialize lists to store formatted data
formatted_data = []
# Set max length for padding
max_length = 20

for index, row in df.iterrows():
    name = row["names"]
    phonetic = row["phonetics"]
    
    # Format input-target pair
    input_sequence = f"Convert name to phonetic pronunciation:{name}"
    target_sequence = phonetic
    
    # Tokenize input and target sequences
    input_tokens = tokenizer.encode(input_sequence, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")[0]
    target_tokens = tokenizer.encode(target_sequence, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")[0]
    
    # Add formatted data to the list
    formatted_data.append({"input_ids": input_tokens, "labels": target_tokens})
print(formatted_data[:50])

[{'input_ids': tensor([ 1193,  3027,   564,    12,   951,  1225, 30637,    10,  2427,  5983,
           76,     1,     0,     0,     0,     0,     0,     0,     0,     0]), 'labels': tensor([3342,  107,   18, 7400,  476,   18,   32,   32,    1,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])}, {'input_ids': tensor([ 1193,  3027,   564,    12,   951,  1225, 30637,    10,    26,   138,
        13272,     1,     0,     0,     0,     0,     0,     0,     0,     0]), 'labels': tensor([10729,    18, 13272,     1,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])}, {'input_ids': tensor([ 1193,  3027,   564,    12,   951,  1225, 30637,    10,  3225,  9629,
            1,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'labels': tensor([    3,    88,   107,    29,    18,    52, 10266,     1,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]

In [6]:
# Split the dataset into training, validation, and test sets
train_data = formatted_data[:int(0.8 * len(formatted_data))]
val_data = formatted_data[int(0.8 * len(formatted_data)):int(0.9 * len(formatted_data))]
test_data = formatted_data[int(0.9 * len(formatted_data)):]


# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    learning_rate=1e-3,
    num_train_epochs=60,
    logging_steps=100,
    logging_dir='C:\\testing\\logs',
    output_dir="C:\\testing\\output",
    weight_decay=0.1,
    
)



# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()

# Evaluate the model
trainer.evaluate()

# Save the fine-tuned model
model.save_pretrained("C:\\testing\\fine_tuned_t5_model")

C:\Users\nehaparveenmohamm\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,1.294700
200,0.752700
300,0.659200
400,0.617400
500,0.570000
600,0.549400
700,0.508200
800,0.467800
900,0.463800
1000,0.454500


In [7]:
model_checkpoint_path = "C:\\testing\\fine_tuned_t5_model"

model = T5ForConditionalGeneration.from_pretrained(model_checkpoint_path)

In [27]:
# Input text
input_text = "Convert name to phonetic pronunciation:Saoirse"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate output
output = model.generate(input_ids)

# Decode the output tokens to text
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Output:", output_text)

Output: seer-sha


In [ ]:
# pip install pandas

In [ ]:
# pip install sentencepiece

In [ ]:
# pip install transformers

In [ ]:
# pip install torch torchvision torchaudio

In [ ]:
# pip install transformers[torch]

In [ ]:
# pip install accelerate -U

In [ ]:
# pip install --upgrade torch torchvision torchaudio transformers

In [ ]:
# pip install protobuf==3.20.0

In [ ]:
# pip install rouge

In [ ]:
# pip install nltk

In [9]:
import torch
from torch import tensor
from transformers import T5Tokenizer, T5ForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge


# Decode predictions
predictions = []
for example in test_data:
    input_ids = example["input_ids"].unsqueeze(0)  # Add batch dimension
    outputs = model.generate(input_ids=input_ids, max_length=20, num_beams=5, early_stopping=True)
    predicted_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(predicted_sequence.lower())


# Prepare references
references = [tokenizer.decode(example["labels"], skip_special_tokens=True).lower() for example in test_data]

In [10]:
f_data = []
for i in range (len(predictions)):
    new_dict = {"predictions":predictions[i],
     "references":references[i]}
    f_data.append(new_dict)


In [11]:
print(f_data[:50])
print(len(f_data))

[{'predictions': 'peeks', 'references': 'peeks'}, {'predictions': 'ah-fon-so', 'references': 'af-ohn-soo'}, {'predictions': 'kah-zoo-mee', 'references': 'kah-zoo-mee'}, {'predictions': 'koh-moh-ros', 'references': 'kah-moh-rohz'}, {'predictions': 'mah-see-ray', 'references': 'mah-see-ray'}, {'predictions': 'eh-roh', 'references': 'eh-roh'}, {'predictions': 'see-boo-see-soh', 'references': 'see-boo-see-soh'}, {'predictions': 'boo-koh-lah', 'references': 'boo-koh-lah'}, {'predictions': 'kah-tar-zee-nah', 'references': 'kah-tar-zih-nah'}, {'predictions': 'raht-see-rahk', 'references': 'raht-see-rahk'}, {'predictions': 'hee-nuh', 'references': 'hee-nah'}, {'predictions': 'oh-mah-ree-nee', 'references': 'oh-mah-ree-nee'}, {'predictions': 'hah-nee-yeh', 'references': 'hah-nee-ah'}, {'predictions': 'shuh-nee-koo', 'references': 'shuh-nee-kwuh'}, {'predictions': 'kyung-ho', 'references': 'kyuhng-ho'}, {'predictions': 'sa-mee', 'references': 'sa-mi'}, {'predictions': 'ray-ed', 'references': 'ra

In [12]:
# Calculate BLEU score
bleu_score = corpus_bleu(list_of_references=[[ref.split("-")] for ref in references], hypotheses=[pred.split("-") for pred in predictions])
print(f"BLEU Score:{bleu_score}")

BLEU Score:0.3482552033765938


In [14]:
# Calculate ROUGE score
rouge = Rouge()
scores = rouge.get_scores(hyps=predictions, refs=references, avg=True)
print(f"ROUGE Scores: {scores}")

ROUGE Scores: {'rouge-1': {'r': 0.4612876935615322, 'p': 0.4612876935615322, 'f': 0.4612876912551063}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.4612876935615322, 'p': 0.4612876935615322, 'f': 0.4612876912551063}}
